In [111]:
import pandas as pd
import subprocess
import json
import xmltodict
import pprint
import numpy as np
import math
# import  aspose.cells 
# from aspose.cells import Workbook

pd.set_option('max_colwidth',250)
pd.set_option('display.max_rows',90)

todosCargos = Contem os cargos de cada comissão

In [223]:
def cargos(lista):
    todosCargos = pd.DataFrame()
    for i,r in lista.iterrows():
        if r["Cargos"] != None:
            for i,cargos in enumerate(r["Cargos"]):
                iteracao = pd.DataFrame.from_dict(cargos,orient='index').T
                iteracao["CodigoColegiado"] = r["CodigoColegiado"]
                todosCargos = pd.concat([iteracao,todosCargos])
                todosCargos["Bancada"] = todosCargos["Bancada"].str.replace("(","")
                todosCargos["Bancada"] = todosCargos["Bancada"].str.replace(")","")
    return todosCargos.reset_index()#.drop("index", axis=1, inplace=True)

In [224]:
comando = 'curl https://legis.senado.leg.br/dadosabertos/dados/ConselhosOrgaosCongresso.xml'
resultado = subprocess.run(comando, shell=True, text=True, capture_output=True)
resultado = resultado.stdout
resultado = json.dumps(xmltodict.parse(resultado))
resultado = pd.read_json(resultado)
resultado = resultado.loc["Colegiados"]["ConselhosOrgaosCongresso"]
resultado = pd.DataFrame(resultado)
lista = pd.DataFrame()
for i,r in resultado.iterrows():
    aux = pd.DataFrame(r.loc["Colegiado"])
    lista = pd.concat([lista, aux], axis=0)
lista = lista.groupby("CodigoColegiado").first().reset_index()
todosCargos = cargos(lista)

C:\Users\kristyan.silva\AppData\Local\Temp\ipykernel_23824\1019879638.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  resultado = pd.read_json(resultado)


In [5]:
lista

,CodigoColegiado,NomeColegiado,SiglaColegiado,Finalidade,DataInicio,QuantidadesMembros,Cargos,Eventos,MembrosBlocoSF,MembrosBlocoCD,EventosProrrogacao,ObservacoesVaga,InformacoesSecretaria,TipoColegiado,DescricaoSubtitulo
0,1916,Representação Brasileira no Parlamento do Mercosul,CPCMS,A Representação Brasileira no Parlamento do Mercosul tem por finalidade ser o órgão de ligação entre o Congresso Nacional e o Parlamento do Mercosul.,02/02/2015,37,"[{'TipoCargo': 'PRESIDENTE', 'CodigoCargo': '1', 'NomeParlamentar': 'Senador Nelsinho Trad', 'CodigoParlamentar': '5985', 'Bancada': '(PSD-MS)'}, {'TipoCargo': '1º VICE-PRESIDENTE', 'CodigoCargo': '4', 'NomeParlamentar': 'Senador Humberto Costa',...","[{'TipoEvento': 'Designação', 'DataEvento': '11/07/2023', 'OrdemImpressao': '1'}, {'TipoEvento': 'Instalação', 'DataEvento': '13/09/2023', 'OrdemImpressao': '2'}]","[{'UnidadePartidaria': 'Bloco Parlamentar Democracia', 'NumeroMembros': '8', 'PartidosBloco': {'Partidos': [{'SiglaPartido': 'UNIÃO'}, {'SiglaPartido': 'MDB'}, {'SiglaPartido': 'PODEMOS'}, {'SiglaPartido': 'PDT'}, {'SiglaPartido': 'PSDB'}]}, 'Mem...","[{'UnidadePartidaria': 'PL', 'NumeroMembros': '10', 'MembrosCD': {'Membro': [{'ProprietarioVaga': 'PL', 'NomeParlamentar': 'Deputado Giovani Cherini', 'CodigoParlamentar': '5336', 'CodigoDeputadoNaCamara': '160673', 'SiglaUf': 'RS', 'Partido': 'P...","{'Leitura': '02/02/2015', 'TipoEvento': {'Membro': [{'TipoEvento': 'Designação', 'DatEvento': '11/07/2023', 'CodigoColegiado': '1916'}, {'TipoEvento': 'Instalação', 'DatEvento': '13/09/2023', 'CodigoColegiado': '1916'}]}}","[{'Linha': '1', 'ObservacaoParlamentar': '11/07/2023: Designado como suplente o Senador Wellington Fagundes, conforme ATCN 49/2023 e Ofício n° 125/2023 do Bloco Vanguarda.'}, {'Linha': '2', 'ObservacaoParlamentar': '11/07/2023: Designado como tit...",Antônio Ferreira Costa Filho,Órgão,None
1,767,Conselho de Comunicação Social,CCS,"O Conselho de Comunicação Social tem como atribuição a realização de estudos, pareceres, recomendações e outras solicitações que lhe forem encaminhadas pelo Congresso Nacional a respeito do Título VIII, Capítulo V, da Constituição Federal (art. 2...",30/12/1991,None,"[{'TipoCargo': 'PRESIDENTE', 'CodigoCargo': '1', 'NomeParlamentar': 'Miguel Matos', 'Bancada': '(-)'}, {'TipoCargo': 'VICE-PRESIDENTE', 'CodigoCargo': '2', 'NomeParlamentar': 'Patricia Blanco', 'Bancada': '(-)'}]","[{'TipoEvento': 'Eleição Geral', 'DataEvento': '05/06/2002', 'OrdemImpressao': '1'}, {'TipoEvento': 'Eleição Geral', 'DataEvento': '22/12/2004', 'OrdemImpressao': '1'}, {'TipoEvento': 'Eleição Geral', 'DataEvento': '17/07/2012', 'OrdemImpressao':...","[{'NumeroMembros': '2', 'MembrosSF': {'Membro': [{'ProprietarioVaga': 'Representante das empresas de rádio (inciso I)', 'NomeParlamentar': 'Flavio Lara Resende', 'TipoVaga': 'Titular', 'NumeroOrdem': '1', 'OrigemParlamentar': 'Congresso'}, {'Prop...",None,{'Leitura': '30/12/1991'},None,SECRETARIA-GERAL DA MESA,Conselho,"Lei nº 8.389, de 1991,\n Regimento Interno aprovado nos termos do Ato da Mesa nº 1, de 2013"
2,768,Conselho da Ordem do Congresso Nacional,COCN,"A Ordem do Congresso Nacional é destinada a prestigiar as pessoas físicas ou jurídicas, nacionais ou estrangeiras, que se tenham tornado dignas do especial reconhecimento do Poder Legislativo do Brasil. Compete ao Conselho aprovar ou rejeitar as ...",23/11/1972,22,None,"[{'TipoEvento': 'Atualização', 'DataEvento': '12/09/2012', 'OrdemImpressao': '8'}, {'TipoEvento': 'Atualização', 'DataEvento': '04/02/2013', 'OrdemImpressao': '8'}, {'TipoEvento': 'Atualização', 'DataEvento': '07/03/2013', 'OrdemImpressao': '8'},...",None,None,{'Leitura': '23/11/1972'},None,"Núcleo de Premiações, Frentes e Grupos Parlamentares - NPFG",Conselho,"Decreto Legislativo nº 70, de 1972, regulamentado pelo Ato nº 1, de 1973-CN"
3,769,Conselho do Diploma do Mérito Educativo Darcy Ribeiro,DMEDR,O Diploma do Mérito Educativo Darcy Ribeiro é entregue pelo Congresso Nacional a pessoas ou

In [194]:
def membros(lista):
    todosMembros = pd.DataFrame()
    for i, r in lista.fillna("").iterrows():
        if r["MembrosBlocoSF"] is not None:
            for membros in r["MembrosBlocoSF"]:
                if membros is not None and "MembrosSF" in membros:
                    iteracao = pd.DataFrame.from_dict(membros, orient='index').T
                    for i,membro in iteracao.iterrows():
                        # print(type(membro["MembrosSF"]["Membro"][0]))
                        iteracaoMembro = pd.DataFrame.from_dict(membro["MembrosSF"]["Membro"][0], orient='index').T
                        iteracaoMembro["CodigoColegiado"] = r["CodigoColegiado"]
                        iteracao = pd.concat([iteracao, iteracaoMembro])
                    todosMembros = pd.concat([iteracao, todosMembros])
    
    return todosMembros[todosMembros["CodigoParlamentar"].notna()].bfill().groupby("NomeParlamentar").first().dropna(axis=1).reset_index()

In [181]:
todosMembros = membros(lista)

In [192]:
# todosMembros = todosMembros[todosMembros["CodigoParlamentar"].notna()].groupby("NomeParlamentar").first().dropna(axis=1).reset_index()
todosMembros2 = todosMembros[todosMembros["CodigoParlamentar"].notna()].bfill().groupby("NomeParlamentar").first().dropna(axis=1).reset_index()

In [193]:
todosMembros2

,NomeParlamentar,ProprietarioVaga,TipoVaga,NumeroOrdem,OrigemParlamentar,CodigoColegiado,CodigoParlamentar,SiglaUf,Partido,Ramais,ObservacaoParlamentar
0,Senador Eduardo Girão,Bloco Parlamentar Vanguarda,Titular,1,Senado,1916,5976,CE,NOVO,6677 / 6679,17
1,Senador Renan Calheiros,Bloco Parlamentar Democracia,Titular,1,Senado,1916,70,AL,MDB,2261 / 2299,2
2,Senador Sérgio Petecão,Bloco Parlamentar da Resistência Democrática,Titular,1,Senado,1916,4560,AC,PSD,4086 / 6709,9
3,Senadora Tereza Cristina,Bloco Parlamentar Aliança,Titular,1,Senado,1916,5736,MS,PP,2431,15


In [225]:
todosCargos

,index,TipoCargo,CodigoCargo,NomeParlamentar,Bancada,CodigoColegiado,CodigoParlamentar,CodigoDeputadoNaCamara
0,0,VICE-PRESIDENTE,2,Patricia Blanco,-,767,NaN,NaN
1,0,PRESIDENTE,1,Miguel Matos,-,767,NaN,NaN
2,0,VICE-PRESIDENTE PELO BRASIL NO PARLASUL,228,Deputado Arlindo Chinaglia,PT-SP,1916,853,73433
3,0,2º VICE-PRESIDENTE,145,Deputado Celso Russomanno,REPUBLICANOS-SP,1916,859,73441
4,0,1º VICE-PRESIDENTE,4,Senador Humberto Costa,PT-PE,1916,5008,NaN
5,0,PRESIDENTE,1,Senador Nelsinho Trad,PSD-MS,1916,5985,NaN
